# Wordle?

> Any CS student should be able to create a wordle solver.

I've lost track of how many said this on Twitter and I completely agree. But as [1blue3brown](https://youtu.be/v68zYyaEmEA) showed in a recent video, there is a information-theoretic upper limit to what the best wordle solver can score, and it generally boils down to the probabilities between the frequency of the particular word appearing in the language and the wordle games selection of words.

But what if all probabilities are equal?

Let's ask the computer to answer this question...

In [5]:
import pathlib
import random
words = pathlib.Path('wordle_words.txt').read_text('utf-8').split('\n')

def play():
    print("(q) to quit, (r) for random word")
    ans = None
    while ans != 'q':
        ans = input("Enter the secret word >>>")
        if ans == 'r':
            ans = random.choice(words)
        elif ans == 'q':
            break
        
        if ans not in words:
            print(f"{ans} is not an english 5 letter word")
        else:
            autowordle(secret_word=ans)

def autowordle(secret_word):
    # computer tries to guess.
    options = words[:]
    while 1:
        guess = random.choice(options)
        if guess == secret_word:
            print(f"{guess} is correct!")
            break
        L = []
        for ix, (a, b) in enumerate(zip(guess, secret_word)):
            if a == b:
                L.append(a.upper())
                options = [w for w in options if w[ix] == a]
            elif a in secret_word:
                L.append(a.lower())
                options = [w for w in options if a in w]
            else:  # a not in b:
                L.append('-')
                options = [w for w in options if a not in w]
        print(guess, "-->", "".join(L))

In [6]:
play()

(q) to quit, (r) for random word
Enter the secret word >>>CRANE
JURDI --> --r--
FREAM --> -Rea-
GRATE --> -RA-E
CRAYE --> CRA-E
CRANE is correct!
Enter the secret word >>>r
BOONE --> -Oo--
SOAPY --> -Oa--
KOKIA --> -O--A
LOTHA --> -O--A
MOCOA --> mO-oA
MOMMA --> mOmMA
DOGMA --> -O-MA
FORMA is correct!
Enter the secret word >>>q


## Here's the normal wordle game ...

In [8]:
import pathlib
import random
words = pathlib.Path('wordle_words.txt').read_text('utf-8').split('\n')

secret_word = random.choice(words)
print("secret word has been chosen. Your turn...")
while 1:
    guess = input(">>>")
    if guess == 'q':
        break
    if guess == 'r':
        guess = random.choice(options)
    
    if guess not in words:
        print(guess, "is not a word")
        continue
        
    if guess == secret_word:
        print(f"{guess} is correct!")
        break
    options = words[:]
    L = []
    for ix, (a, b) in enumerate(zip(guess, secret_word)):
        if a == b:
            L.append(a.upper())
            options = [w for w in options if w[ix]==a]
        elif a in secret_word:
            L.append(a.lower())
            options = [w for w in options if a in w]
        else:  # a not in b:
            L.append("-")
            options = [w for w in options if a not in w]

    print(guess, "-->","".join(L))

secret word has been chosen. Your turn...
>>>CRANE
CRANE --> c----
>>>r
OPHIC --> ---ic
>>>r
DNCRI --> --C-i
>>>r
PICKS --> -IC-S
>>>r
RICES --> -IC-S
>>>r
TICKS --> -IC-S
>>>r
MICAS --> -IC-S
>>>r
DICES --> -IC-S
>>>r
RICKS --> -IC-S
>>>r
BICES --> -IC-S
>>>r
NICKS --> -IC-S
>>>r
SICES --> sIC-S
>>>r
OICKS --> -IC-S
>>>r
PICAS --> -IC-S
>>>r
SICKS --> sIC-S
>>>r
VICES --> -IC-S
>>>r
WICKS --> -IC-S
>>>r
VICUS --> -ICUS
>>>r
FICUS is correct!
